In [13]:
# from dask.distributed import Client

# client = Client("tcp://10.32.0.154:42411")
# client

In [14]:
%conda install -c conda-forge git+https://github.com/mathause/regionmask.git
%conda install -c conda-forge git+https://github.com/jbusecke/cmip6_preprocessing.git


Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.

PackagesNotFoundError: The following packages are not available from current channels:

  - //github.com/mathause/regionmask.git

Current channels:

  - https://conda.anaconda.org/conda-forge/osx-64
  - https://conda.anaconda.org/conda-forge/noarch
  - https://repo.anaconda.com/pkgs/main/osx-64
  - https://repo.anaconda.com/pkgs/main/noarch
  - https://repo.anaconda.com/pkgs/r/osx-64
  - https://repo.anaconda.com/pkgs/r/noarch

To search for alternate channels that may provide the conda package you're
looking for, navigate to

    https://anaconda.org

and use the search bar at the top of the page.



Note: you may need to restart the kernel to use updated packages.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with initial frozen solve. Retrying with 

Lets import ocean temperature data from a variety of models to see if the masking works well.

In [15]:
import regionmask
import intake
import matplotlib.pyplot as plt
from cmip6_preprocessing.preprocessing import combined_preprocessing
import xarray as xr
import numpy as np

In [17]:
# import example cloud datasets
col_url = "https://raw.githubusercontent.com/NCAR/intake-esm-datastore/master/catalogs/pangeo-cmip6.json"
col = intake.open_esm_datastore(col_url)
cat = col.search(source_id=['CAMS-CSM1-0', 'CNRM-CM6-1', 'CNRM-ESM2-1', 'ACCESS-CM2', 'ACCESS-ESM1-5', 'EC-Earth3-Veg',
                            'MIROC-ES2L', 'MIROC6', 'HadGEM3-GC31-LL', 'UKESM1-0-LL', 'MPI-ESM1-2-HR', 'MRI-ESM2-0',
                            'NorCPM1', 'GFDL-CM4', 'GFDL-ESM4', 'NESM3'],
                 experiment_id='historical', variable_id='thetao')
data_dict = cat.to_dataset_dict(zarr_kwargs={'consolidated': True, 'decode_times': False},
                                preprocess=combined_preprocessing)

<function combined_preprocessing at 0x18aa31af0> combined_preprocessing print

--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'


RuntimeError: Failed to apply pre-processing function: combined_preprocessing

First we need to load the natural earth ocean basins.

In [ ]:
# load ocean basin data
basins = regionmask.defined_regions.natural_earth.ocean_basins_50
basins.plot(add_ocean=False, add_label=False)

In [ ]:
from cmip6_preprocessing.regionmask import merged_mask

When using the defaults this will index the North and South Pacific with numeric values of 2 and 3 respectively. The Maritime Continent gets a separate index (4). We will now load example data from many CMIP6 models, create the mask, and select only the points with values 2,3, and 4.

In [ ]:
# loop over models and mask out the pacific
import cartopy.crs as ccrs

for k, ds in data_dict.items():
    if 'lev' in ds.dims:
        model = ds.attrs['source_id']
        if 'member_id' in ds.dims:
            ds = ds.isel(member_id=0)
        ds = ds.thetao.isel(time=0, lev=0).squeeze()

        mask = merged_mask(basins,ds)

        kwargs = dict(x='lon', y='lat',transform = ccrs.PlateCarree(), infer_intervals=False)
        fig, (ax1, ax2, ax3) = plt.subplots(ncols=3, figsize=[20,2], subplot_kw={'projection':ccrs.Robinson(190)})
        ds.plot(ax=ax1, **kwargs)
        ax1.set_title(f"Raw field {model}")
        
        ds_masked = ds.where(np.logical_or(np.logical_or(mask == 2, mask==3),mask==4)) # Pacific + Maritime Continent
        ds_masked.plot(ax=ax2, **kwargs)
        ax2.set_title(f"Masked Pacific {model}")
        mask.plot(ax=ax3, cmap='tab20', vmin=0, vmax=19, **kwargs)
        ax3.set_title(f"Full Mask {model}")
        for ax in [ax1, ax2, ax3]:
            ax.coastlines()